***GENERATED CODE FOR gettingthedata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order ID', 'transformation_label': 'String Indexer'}], 'feature': 'Order ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '41032', 'mean': '', 'stddev': '', 'min': 'AE-2012-PO8865138-41184', 'max': 'ZA-2015-TS11205146-42062', 'missing': '0', 'distinct': '22386'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order ID'}, {'feature_label': 'Order ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order Date', 'transformation_label': 'String Indexer'}], 'feature': 'Order Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': '1/1/2012', 'max': '9/9/2015', 'missing': '0', 'distinct': '1427'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order Date'}, {'feature_label': 'Order Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ship Date', 'transformation_label': 'String Indexer'}], 'feature': 'Ship Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': '1/1/2013', 'max': '9/9/2015', 'missing': '0', 'distinct': '1464'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ship Date'}, {'feature_label': 'Ship Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ship Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ship Mode', 'transformation_label': 'String Indexer'}], 'feature': 'Ship Mode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'First Class', 'max': 'Standard Class', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ship Mode'}, {'feature_label': 'Ship Mode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ship Mode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'}], 'feature': 'Customer ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'AA-10315102', 'max': 'ZD-2192596', 'missing': '0', 'distinct': '15677'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer ID'}, {'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Aaron Bergman', 'max': 'Zuschuss Donatelli', 'missing': '0', 'distinct': '796'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Name'}, {'feature_label': 'Customer Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Home Office', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Segment'}, {'feature_label': 'Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Aachen', 'max': 'Águas Lindas de Goiás', 'missing': '0', 'distinct': '3527'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': "'Ajman", 'max': 'Žilina', 'missing': '0', 'distinct': '1085'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Afghanistan', 'max': 'Zimbabwe', 'missing': '0', 'distinct': '162'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Region', 'transformation_label': 'String Indexer'}], 'feature': 'Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Canada', 'max': 'Western US', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Region'}, {'feature_label': 'Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Market', 'transformation_label': 'String Indexer'}], 'feature': 'Market', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Africa', 'max': 'USCA', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Market'}, {'feature_label': 'Market', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Market')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product ID', 'transformation_label': 'String Indexer'}], 'feature': 'Product ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'FUR-BO-3174', 'max': 'TEC-PH-6626', 'missing': '0', 'distinct': '3752'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product ID'}, {'feature_label': 'Product ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category', 'transformation_label': 'String Indexer'}], 'feature': 'Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category'}, {'feature_label': 'Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': 'Accessories', 'max': 'Tables', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sub-Category'}, {'feature_label': 'Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Name', 'transformation_label': 'String Indexer'}], 'feature': 'Product Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '', 'stddev': '', 'min': '"""While you Were Out"" Message Book', 'max': 'netTALK DUO VoIP Telephone Service', 'missing': '0', 'distinct': '3760'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Name'}, {'feature_label': 'Product Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sales', 'transformation_label': 'String Indexer'}], 'feature': 'Sales', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '268.23', 'stddev': '523.75', 'min': ' 10/Pack"', 'max': '999.98', 'missing': '0', 'distinct': '20197'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sales'}, {'feature_label': 'Sales', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sales')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Quantity', 'transformation_label': 'String Indexer'}], 'feature': 'Quantity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '3.9', 'stddev': '11.97', 'min': ' 1040 sheets"', 'max': '98.352', 'missing': '0', 'distinct': '186'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Quantity'}, {'feature_label': 'Quantity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Quantity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Discount', 'transformation_label': 'String Indexer'}], 'feature': 'Discount', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '0.18', 'stddev': '1.63', 'min': ' 30/Box"', 'max': '98.352', 'missing': '0', 'distinct': '48'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Discount'}, {'feature_label': 'Discount', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Discount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order Priority', 'transformation_label': 'String Indexer'}], 'feature': 'Order Priority', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '6.84', 'stddev': '13.76', 'min': '-25.8174', 'max': 'Medium', 'missing': '0', 'distinct': '193'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order Priority'}, {'feature_label': 'Order Priority', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order Priority')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Returned', 'transformation_label': 'String Indexer'}], 'feature': 'Returned', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '72.56', 'stddev': '99.74', 'min': '1.14', 'max': 'Yes', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Returned'}, {'feature_label': 'Returned', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Returned')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Person', 'transformation_label': 'String Indexer'}], 'feature': 'Person', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '41032', 'mean': '20.14', 'stddev': '19.16', 'min': '1.52', 'max': 'Yes', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Person'}, {'feature_label': 'Person', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Person')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run gettingthedataHooks.ipynb
try:
	#sourcePreExecutionHook()

	mergedfinaldatasetglobalsuperstoresales = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Merged_FinalDataset_GlobalSuperStoreSales.csv', 'filename': 'Merged_FinalDataset_GlobalSuperStoreSales.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gettingthedataHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofedata = TransformationMain.run(mergedfinaldatasetglobalsuperstoresales,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Order ID", "transformation_label": "String Indexer"}], "feature": "Order ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "AE-2012-PO8865138-41184", "max": "ZA-2015-TS11205146-42062", "missing": "0", "distinct": "22386"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order ID"}, {"transformationsData": [{"feature_label": "Order Date", "transformation_label": "String Indexer"}], "feature": "Order Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "1/1/2012", "max": "9/9/2015", "missing": "0", "distinct": "1427"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order Date"}, {"transformationsData": [{"feature_label": "Ship Date", "transformation_label": "String Indexer"}], "feature": "Ship Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "1/1/2013", "max": "9/9/2015", "missing": "0", "distinct": "1464"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ship Date"}, {"transformationsData": [{"feature_label": "Ship Mode", "transformation_label": "String Indexer"}], "feature": "Ship Mode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "First Class", "max": "Standard Class", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode"}, {"transformationsData": [{"feature_label": "Customer ID", "transformation_label": "String Indexer"}], "feature": "Customer ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "AA-10315102", "max": "ZD-2192596", "missing": "0", "distinct": "15677"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer ID"}, {"transformationsData": [{"feature_label": "Customer Name", "transformation_label": "String Indexer"}], "feature": "Customer Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Aaron Bergman", "max": "Zuschuss Donatelli", "missing": "0", "distinct": "796"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Name"}, {"transformationsData": [{"feature_label": "Segment", "transformation_label": "String Indexer"}], "feature": "Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Consumer", "max": "Home Office", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Segment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Postal Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41032", "mean": "55468.43", "stddev": "14222.31", "min": "1040", "max": "99301", "missing": "0"}, "updatedLabel": "Postal Code"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Aachen", "max": "\u00c1guas Lindas de Goi\u00e1s", "missing": "0", "distinct": "3527"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "'Ajman", "max": "\u017dilina", "missing": "0", "distinct": "1085"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Afghanistan", "max": "Zimbabwe", "missing": "0", "distinct": "162"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": "Region", "transformation_label": "String Indexer"}], "feature": "Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Canada", "max": "Western US", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Region"}, {"transformationsData": [{"feature_label": "Market", "transformation_label": "String Indexer"}], "feature": "Market", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Africa", "max": "USCA", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Market"}, {"transformationsData": [{"feature_label": "Product ID", "transformation_label": "String Indexer"}], "feature": "Product ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "FUR-BO-3174", "max": "TEC-PH-6626", "missing": "0", "distinct": "3752"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product ID"}, {"transformationsData": [{"feature_label": "Category", "transformation_label": "String Indexer"}], "feature": "Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category"}, {"transformationsData": [{"feature_label": "Sub-Category", "transformation_label": "String Indexer"}], "feature": "Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "Accessories", "max": "Tables", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sub-Category"}, {"transformationsData": [{"feature_label": "Product Name", "transformation_label": "String Indexer"}], "feature": "Product Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "", "stddev": "", "min": "\"\"\"While you Were Out\"\" Message Book", "max": "netTALK DUO VoIP Telephone Service", "missing": "0", "distinct": "3760"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Name"}, {"transformationsData": [{"feature_label": "Sales", "transformation_label": "String Indexer"}], "feature": "Sales", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "268.23", "stddev": "523.75", "min": " 10/Pack\"", "max": "999.98", "missing": "0", "distinct": "20197"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"feature_label": "Quantity", "transformation_label": "String Indexer"}], "feature": "Quantity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "3.9", "stddev": "11.97", "min": " 1040 sheets\"", "max": "98.352", "missing": "0", "distinct": "186"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Quantity"}, {"transformationsData": [{"feature_label": "Discount", "transformation_label": "String Indexer"}], "feature": "Discount", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "0.18", "stddev": "1.63", "min": " 30/Box\"", "max": "98.352", "missing": "0", "distinct": "48"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "31.21", "stddev": "185.19", "min": "-6599.0", "max": "8399.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Shipping Cost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "27.83", "stddev": "61.78", "min": "-292.0", "max": "933.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Shipping Cost"}, {"transformationsData": [{"feature_label": "Order Priority", "transformation_label": "String Indexer"}], "feature": "Order Priority", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "6.84", "stddev": "13.76", "min": "-25.8174", "max": "Medium", "missing": "0", "distinct": "193"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order Priority"}, {"transformationsData": [{"feature_label": "Returned", "transformation_label": "String Indexer"}], "feature": "Returned", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "72.56", "stddev": "99.74", "min": "1.14", "max": "Yes", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Returned"}, {"transformationsData": [{"feature_label": "Person", "transformation_label": "String Indexer"}], "feature": "Person", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "41032", "mean": "20.14", "stddev": "19.16", "min": "1.52", "max": "Yes", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Person"}]}))

	#transformationPostExecutionHook(autofedata)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run gettingthedataHooks.ipynb
try:
	#sinkPreExecutionHook()

	globalsuperstoresalesoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/6610a445002ba9607526ef6e/part-00000-4828b373-7d3b-48f3-9359-a429c8831281-c000.csv', 'samplecount': 5137, 'originalcount': 51290, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Merged_FinalDataset_GlobalSuperStoreSales.csv', 'pathOnly': '/Global SuperStore Sales 2016', 'project_id': '6610a2c8002ba9607526ef57', 'parent_id': '6610a2c8002ba9607526ef57', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58791', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58792', 'field': 'Profit', 'alias': 'Profit', 'type': 'real', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58793', 'field': 'Shipping Cost', 'alias': 'Shipping Cost', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58794', 'field': 'Order ID_stringindexer', 'alias': 'Order ID_stringindexer', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58795', 'field': 'Order Date_stringindexer', 'alias': 'Order Date_stringindexer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58796', 'field': 'Ship Date_stringindexer', 'alias': 'Ship Date_stringindexer', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58797', 'field': 'Ship Mode_stringindexer', 'alias': 'Ship Mode_stringindexer', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58798', 'field': 'Customer ID_stringindexer', 'alias': 'Customer ID_stringindexer', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58799', 'field': 'Customer Name_stringindexer', 'alias': 'Customer Name_stringindexer', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879a', 'field': 'Segment_stringindexer', 'alias': 'Segment_stringindexer', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879b', 'field': 'City_stringindexer', 'alias': 'City_stringindexer', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879c', 'field': 'State_stringindexer', 'alias': 'State_stringindexer', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879d', 'field': 'Country_stringindexer', 'alias': 'Country_stringindexer', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879e', 'field': 'Region_stringindexer', 'alias': 'Region_stringindexer', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5879f', 'field': 'Market_stringindexer', 'alias': 'Market_stringindexer', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a0', 'field': 'Product ID_stringindexer', 'alias': 'Product ID_stringindexer', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a1', 'field': 'Category_stringindexer', 'alias': 'Category_stringindexer', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a2', 'field': 'Sub-Category_stringindexer', 'alias': 'Sub-Category_stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a3', 'field': 'Product Name_stringindexer', 'alias': 'Product Name_stringindexer', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a4', 'field': 'Sales_stringindexer', 'alias': 'Sales_stringindexer', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a5', 'field': 'Quantity_stringindexer', 'alias': 'Quantity_stringindexer', 'type': 'real', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a6', 'field': 'Discount_stringindexer', 'alias': 'Discount_stringindexer', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a7', 'field': 'Order Priority_stringindexer', 'alias': 'Order Priority_stringindexer', 'type': 'real', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a8', 'field': 'Returned_stringindexer', 'alias': 'Returned_stringindexer', 'type': 'numeric', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e587a9', 'field': 'Person_stringindexer', 'alias': 'Person_stringindexer', 'type': 'real', 'position': '24'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58778', 'field': 'Order ID', 'alias': 'Order ID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58779', 'field': 'Order Date', 'alias': 'Order Date', 'type': 'date', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877a', 'field': 'Ship Date', 'alias': 'Ship Date', 'type': 'date', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877b', 'field': 'Ship Mode', 'alias': 'Ship Mode', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877c', 'field': 'Customer ID', 'alias': 'Customer ID', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877d', 'field': 'Customer Name', 'alias': 'Customer Name', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877e', 'field': 'Segment', 'alias': 'Segment', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5877f', 'field': 'Postal Code', 'alias': 'Postal Code', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58780', 'field': 'City', 'alias': 'City', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58781', 'field': 'State', 'alias': 'State', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58782', 'field': 'Country', 'alias': 'Country', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58783', 'field': 'Region', 'alias': 'Region', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58784', 'field': 'Market', 'alias': 'Market', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58785', 'field': 'Product ID', 'alias': 'Product ID', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58786', 'field': 'Category', 'alias': 'Category', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58787', 'field': 'Sub-Category', 'alias': 'Sub-Category', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58788', 'field': 'Product Name', 'alias': 'Product Name', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58789', 'field': 'Sales', 'alias': 'Sales', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878a', 'field': 'Quantity', 'alias': 'Quantity', 'type': 'String', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878b', 'field': 'Discount', 'alias': 'Discount', 'type': 'String', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878c', 'field': 'Profit', 'alias': 'Profit', 'type': 'real', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878d', 'field': 'Shipping Cost', 'alias': 'Shipping Cost', 'type': 'real', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878e', 'field': 'Order Priority', 'alias': 'Order Priority', 'type': 'String', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e5878f', 'field': 'Returned', 'alias': 'Returned', 'type': 'String', 'position': '23'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '66118943365a418603e58790', 'field': 'Person', 'alias': 'Person', 'type': 'String', 'position': '24'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'CleanedData.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-7561895a-9940-4f80-a05b-28ed5d9ce0d7-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
